![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

<div style="text-align: center;">
    <h2>Write by Hyonta B</h2>
</div>

<a href="https://grouplens.org/datasets/movielens/" target="_parent">
  <img src="https://www.svgrepo.com/show/179208/open-sign.svg" alt="Dataset MovieLens" style="width: 50px; height: 50px;">
  open and dwonlode dataset
</a>


### Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV

# Chargement des fichiers CSV
movies = pd.read_csv('https://raw.githubusercontent.com/hyontnick/model_recomendation/main/data/movies.csv').head(60000)
ratings = pd.read_csv('https://raw.githubusercontent.com/hyontnick/model_recomendation/main/data/ratings.csv').head(60000)



# Fusionner les données sur 'movieId'
data = pd.merge(ratings, movies, on='movieId')

# Créer la colonne genres_encoded
le = LabelEncoder()
data['genres_encoded'] = le.fit_transform(data['genres'])

# Créer dataset_copy pour les étapes suivantes
dataset_copy = data[['userId', 'genres_encoded', 'rating']]

# Normaliser les colonnes 'userId' et 'genres_encoded'
scaler = MinMaxScaler()
dataset_copy[['userId', 'genres_encoded']] = scaler.fit_transform(dataset_copy[['userId', 'genres_encoded']])

# Charger le dataset pour Surprise
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(dataset_copy[['userId', 'genres_encoded', 'rating']], reader)

# Diviser les données en ensembles d'entraînement et de test pour Surprise
trainset, testset = train_test_split(surprise_data, test_size=0.25)

# Définir les paramètres à tester pour la recherche en grille
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.007],
    'reg_all': [0.02, 0.04, 0.06]
}

# Faire une recherche en grille pour trouver les meilleurs hyperparamètres
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(surprise_data)

# Meilleurs paramètres trouvés
best_params = gs.best_params['rmse']
print(f"Best parameters: {best_params}")

# Entraîner le modèle SVD avec les meilleurs paramètres
best_svd = SVD(n_factors=best_params['n_factors'], n_epochs=best_params['n_epochs'],
               lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
best_svd.fit(trainset)

# Faire des prédictions avec le modèle optimisé
svd_predictions_optimized = best_svd.test(testset)

# Fonction pour faire une prédiction basée sur le contenu
genre_means = data.groupby('genres_encoded')['rating'].mean().to_dict()

def content_based_prediction(user_id, genres_encoded):
    return genre_means.get(genres_encoded, 3.0)  # Retourner une note moyenne ou 3.0 par défaut

# Combiner les prédictions des deux modèles (SV optimisé et modèle basé sur le contenu)
hybrid_predictions_optimized = []
for pred in svd_predictions_optimized:
    content_pred = content_based_prediction(pred.uid, pred.iid)
    hybrid_pred = (pred.est + content_pred) / 2  # Combiner en moyenne simple
    hybrid_predictions_optimized.append((pred.uid, pred.iid, hybrid_pred, pred.r_ui))

# Calculer le RMSE pour les prédictions hybrides optimisées
true_ratings_optimized = [pred[3] for pred in hybrid_predictions_optimized]
pred_ratings_optimized = [pred[2] for pred in hybrid_predictions_optimized]
rmse_optimized = mean_squared_error(true_ratings_optimized, pred_ratings_optimized, squared=False)
print(f"Hybrid RMSE (Optimized): {rmse_optimized}")

# Calculer MAE pour les prédictions hybrides
mae_optimized = mean_absolute_error(true_ratings_optimized, pred_ratings_optimized)
print(f"Hybrid MAE (Optimized): {mae_optimized}")

# Fonction pour calculer Precision@k et Recall@k pour les prédictions hybrides
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = {}
    for pred in predictions:
        user_est_true.setdefault(pred[0], []).append((pred[2], pred[3]))

    precisions = []
    recalls = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1)
        recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 1)

    precision = np.mean(precisions)
    recall = np.mean(recalls)
    return precision, recall

# Calculer Precision@k et Recall@k pour les prédictions hybrides optimisées
precision_optimized, recall_optimized = precision_recall_at_k(hybrid_predictions_optimized, k=10)
print(f"Hybrid Precision@10 (Optimized): {precision_optimized}")
print(f"Hybrid Recall@10 (Optimized): {recall_optimized}")

# Fonction pour calculer NDCG@k pour les prédictions hybrides
def ndcg_at_k(predictions, k=10):
    def dcg(relevances, k):
        return sum((rel / np.log2(idx + 2) for idx, rel in enumerate(relevances[:k])))

    user_est_true = {}
    for pred in predictions:
        user_est_true.setdefault(pred[0], []).append((pred[2], pred[3]))

    ndcg_values = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        true_relevances = [true_r for (_, true_r) in user_ratings]
        est_relevances = [est for (est, _) in user_ratings]

        idcg = dcg(sorted(true_relevances, reverse=True), k)
        dcg_value = dcg(est_relevances, k)

        ndcg = dcg_value / idcg if idcg != 0 else 0
        ndcg_values.append(ndcg)

    return np.mean(ndcg_values)

# Calculer NDCG@k pour les prédictions hybrides optimisées
ndcg_optimized = ndcg_at_k(hybrid_predictions_optimized, k=10)
print(f"Hybrid NDCG@10 (Optimized): {ndcg_optimized}")

# Fonction pour calculer MAP@k pour les prédictions hybrides
def map_at_k(predictions, k=10):
    user_est_true = {}
    for pred in predictions:
        user_est_true.setdefault(pred[0], []).append((pred[2], pred[3]))

    map_values = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        true_positives = 0
        precisions = []
        for idx, (est, true_r) in enumerate(user_ratings[:k], 1):
            if true_r >= 3.5:
                true_positives += 1
                precisions.append(true_positives / idx)
        if precisions:
            map_values.append(np.mean(precisions))
        else:
            map_values.append(0)

    return np.mean(map_values)

# Calculer MAP@k pour les prédictions hybrides optimisées
mapk_optimized = map_at_k(hybrid_predictions_optimized, k=10)
print(f"Hybrid MAP@10 (Optimized): {mapk_optimized}")


### Parametre

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
import pickle

# Chargement des fichiers CSV
movies = pd.read_csv('ml-25m/movies.csv').head(60000)
ratings = pd.read_csv('ml-25m/ratings.csv').head(60000)

# Fusionner les données sur 'movieId'
data = pd.merge(ratings, movies, on='movieId')

# Créer la colonne genres_encoded
le = LabelEncoder()
data['genres_encoded'] = le.fit_transform(data['genres'])

# Créer dataset_copy pour les étapes suivantes
dataset_copy = data[['userId', 'genres_encoded', 'rating']]

# Normaliser les colonnes 'userId' et 'genres_encoded'
scaler = MinMaxScaler()
dataset_copy[['userId', 'genres_encoded']] = scaler.fit_transform(dataset_copy[['userId', 'genres_encoded']])

# Charger le dataset pour Surprise
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(dataset_copy[['userId', 'genres_encoded', 'rating']], reader)

# Diviser les données en ensembles d'entraînement et de test pour Surprise
trainset, testset = train_test_split(surprise_data, test_size=0.25)

# Définir les paramètres à tester pour la recherche en grille
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.007],
    'reg_all': [0.02, 0.04, 0.06]
}

# Faire une recherche en grille pour trouver les meilleurs hyperparamètres
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(surprise_data)

# Meilleurs paramètres trouvés
best_params = gs.best_params['rmse']
print(f"Best parameters: {best_params}")

# Entraîner le modèle SVD avec les meilleurs paramètres
best_svd = SVD(n_factors=best_params['n_factors'], n_epochs=best_params['n_epochs'],
               lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
best_svd.fit(trainset)

# Sauvegarder le modèle SVD entraîné dans un fichier pickle
with open('best_svd_model.pkl', 'wb') as model_file:
    pickle.dump(best_svd, model_file)

# Calculer les moyennes des genres pour le modèle basé sur le contenu
genre_means = data.groupby('genres_encoded')['rating'].mean().to_dict()

# Sauvegarder les moyennes des genres dans un fichier pickle
with open('genre_means.pkl', 'wb') as genre_file:
    pickle.dump(genre_means, genre_file)

print("Les modèles ont été sauvegardés avec succès.")

### Prediction

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from surprise import SVD

# Charger les fichiers sauvegardés
with open('best_svd_model.pkl', 'rb') as model_file:
    best_svd = pickle.load(model_file)

with open('genre_means.pkl', 'rb') as genre_file:
    genre_means = pickle.load(genre_file)

# Fonction pour faire une prédiction basée sur le contenu
def content_based_prediction(user_id, genres_encoded):
    return genre_means.get(genres_encoded, 3.0)  # Retourner une note moyenne ou 3.0 par défaut

# Fonction pour faire des prédictions hybrides
def hybrid_prediction(user_id, genres_encoded):
    svd_pred = best_svd.predict(user_id, genres_encoded).est
    content_pred = content_based_prediction(user_id, genres_encoded)
    hybrid_pred = (svd_pred + content_pred) / 2
    return hybrid_pred

# Exemple d'utilisation du script de prédiction
if __name__ == "__main__":
    # Charger les données (ici, on utilise un échantillon pour la démonstration)
    ratings = pd.read_csv('ml-25m/ratings.csv').head(60000)
    movies = pd.read_csv('ml-25m/movies.csv').head(60000)
    data = pd.merge(ratings, movies, on='movieId')
    le = LabelEncoder()
    data['genres_encoded'] = le.fit_transform(data['genres'])
    
    # Normaliser les colonnes 'userId' et 'genres_encoded'
    scaler = MinMaxScaler()
    data[['userId', 'genres_encoded']] = scaler.fit_transform(data[['userId', 'genres_encoded']])
    
    user_id = int(input("Entrez l'ID de l'utilisateur: "))
    movie_id = int(input("Entrez l'ID du film: "))
    
    genres_encoded = data[data['movieId'] == movie_id]['genres_encoded'].values[0]
    
    prediction = hybrid_prediction(user_id, genres_encoded)
    print(f"La prédiction hybride pour l'utilisateur {user_id} et le film {movie_id} est {prediction:.2f}")
